In [4]:
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import collections
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
def KNNPCA(UserName,Kv=3,Ka=5,Components=10,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ArousalEncoded'+".csv")
    ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceEncoded'+".csv")
    ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test , z_train,z_test= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
       	
    # KNN classsifier
    clf = KNeighborsClassifier(n_neighbors=Kv)
    trained_model=clf.fit(X_train,y_train)
    trained_model.fit(X_train,y_train )
    
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
        
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
        	
    # KNN classsifier
    clf = KNeighborsClassifier(n_neighbors=Ka)
    trained_model=clf.fit(X_train,z_train)
    trained_model.fit(X_train,z_train )
    clf.fit(X_train, z_train)
    z_predict = clf.predict(X_test)
    return [accuracy_score(z_test, z_predict)*100,accuracy_score(y_test, y_predict)*100
            ,f1_score(z_test, z_predict, average='weighted'),
            f1_score(y_test, y_predict, average='weighted')]

############ Happy And Sad########################33
def KNNPCAHS(UserName,Kv=3,Ka=5,Components=10,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'HappyAndSadEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'HS'+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    while ((np.sum(y_train == 0) == 0) or (np.sum(y_train == 1) == 0)): #Don't Start training unless you have 2 classes.. happens with imbalance
        print("here")
        X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    # KNN classsifier
    clf = KNeighborsClassifier(n_neighbors=Kv)
    trained_model=clf.fit(X_train,y_train)
    trained_model.fit(X_train,y_train )
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]


############ All Emotions########################33
def KNNPCACombined(UserName,Kv=3,Ka=5,Components=10,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceAndArousalEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    # KNN classsifier
    clf = KNeighborsClassifier(n_neighbors=Kv)
    trained_model=clf.fit(X_train,y_train)
    trained_model.fit(X_train,y_train )
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]

################ Run Each Method Multiple Times #####################
def Run100(UserName,Kv=3,Ka=5,Components=10,Window=256,StartTime=0,EndTime=63):
    arousal=[]
    valence=[]
    F1_ScoreArousal=[]
    F1_ScoreValence=[]
    for i in range (100):
        KNN=KNNPCA(UserName,Kv,Ka,Components,Window,StartTime,EndTime)
        arousal+=[KNN[0]]
        valence+=[KNN[1]]
        F1_ScoreArousal+=[KNN[2]]
        F1_ScoreValence+=[KNN[3]]
    print("Arousal "+str(np.mean(arousal)))
    print(collections.Counter(arousal))
    print("Valence "+str(np.mean(valence)))
    print(collections.Counter(valence))
    return [np.mean(arousal),np.mean(valence),np.mean(F1_ScoreArousal),np.mean(F1_ScoreValence)];
    

    
def RunCombined100(UserName,Kv=3,Ka=5,Components=10,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_ALL=[]
    for i in range (100):
        AllEmotions=KNNPCACombined(UserName,Kv,Ka,Components,Window,StartTime,EndTime)
        Combined+=[AllEmotions[0]]
        F1_ALL+=[AllEmotions[1]]
    print("Array of average All Emotions accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_ALL)];
    
def RunHS100(UserName,Kv=3,Ka=5,Components=10,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_HS=[]
    for i in range (100):
        HS=KNNPCAHS(UserName,Kv,Ka,Components,Window,StartTime,EndTime)
        Combined+=[HS[0]]
        F1_HS+=[HS[1]]
    print("Array of average Happy Sad accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_HS)];

##################Main#########################
if __name__ == "__main__":
    HSAccurecy=0;
    HSF1=0;
    CombinedAccurecy=0;
    CombinedF1=0;
    ValenceAccurecy=0;
    ValenceF1=0;
    ArousalAccurecy=0;
    ArousalF1=0;
    for i in range(1,10):
        print("s0"+str(i))
        #Happy and Sad
        var =RunHS100("s0"+str(i))
        print("Happy Sad")
        print(var)
        HSAccurecy+=var[0]
        HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s0"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s0"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
        ValenceF1+=var2[3]
    for i in range(10,33):
        print("s"+str(i))
        #Happy and Sad
        if not ("s"+str(i) == "s31"):
            var =RunHS100("s"+str(i))
            print("Happy Sad")
            print(var)
            HSAccurecy+=var[0]
            HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
    print("Accurecy happy sad for 31 users "+str(HSAccurecy/31))
    print("F1 happy sad for  31 users "+str(HSF1/31))
    print("Accurecy Combined sad for 32 users"+str(CombinedAccurecy/32))
    print("F1 Combined sad for  32 users "+str(CombinedF1/32))
    print("Accurecy Valence for  32 users "+str(ValenceAccurecy/32))
    print("F1 Valence for 32 users "+str(ValenceF1/32))
    print("Accurecy Arousal for  32 users "+str(ArousalAccurecy/32))
    print("F1 Arousal for  32 users "+str(ArousalF1/32))


s01
Array of average Happy Sad accurecies for user s01
Counter({40.0: 40, 60.0: 28, 20.0: 18, 80.0: 9, 0.0: 4, 100.0: 1})
Happy Sad
[44.6, 0.431952380952381]
Array of average All Emotions accurecies for user s01
Counter({25.0: 28, 37.5: 27, 50.0: 21, 12.5: 12, 62.5: 9, 0.0: 2, 75.0: 1})
Combined Emotions
[35.5, 0.3158242243867244]
Arousal 49.0
Counter({50.0: 30, 62.5: 21, 37.5: 20, 25.0: 12, 75.0: 11, 12.5: 3, 87.5: 2, 0.0: 1})
Valence 48.75
Counter({50.0: 32, 37.5: 24, 62.5: 24, 25.0: 9, 75.0: 7, 12.5: 3, 87.5: 1})
Arousal Valence
[49.0, 48.75, 0.45606421356421356, 0.4753245643245645]
s02
Array of average Happy Sad accurecies for user s02
Counter({80.0: 50, 60.0: 23, 100.0: 22, 40.0: 4, 20.0: 1})
Happy Sad
[77.6, 0.7371746031746033]
Array of average All Emotions accurecies for user s02
Counter({37.5: 29, 50.0: 25, 25.0: 19, 62.5: 17, 75.0: 6, 12.5: 3, 87.5: 1})
Combined Emotions
[44.5, 0.3797475302475303]
Arousal 62.75
Counter({62.5: 39, 75.0: 24, 50.0: 15, 87.5: 10, 37.5: 9, 25.0: 3}

Array of average Happy Sad accurecies for user s15
Counter({50.0: 44, 75.0: 31, 25.0: 19, 0.0: 4, 100.0: 2})
Happy Sad
[52.0, 0.49933333333333335]
Array of average All Emotions accurecies for user s15
Counter({37.5: 35, 25.0: 29, 12.5: 16, 50.0: 12, 62.5: 5, 0.0: 2, 75.0: 1})
Combined Emotions
[32.25, 0.3059781746031746]
Arousal 47.5
Counter({37.5: 30, 50.0: 28, 62.5: 20, 25.0: 14, 75.0: 6, 87.5: 2})
Valence 64.0
Counter({62.5: 34, 50.0: 25, 75.0: 24, 87.5: 11, 37.5: 3, 25.0: 2, 100.0: 1})
Arousal Valence
[47.5, 64.0, 0.45555988455988455, 0.6299243256743257]
s16
Array of average Happy Sad accurecies for user s16
Counter({80.0: 54, 100.0: 26, 60.0: 15, 40.0: 4, 20.0: 1})
Happy Sad
[80.0, 0.7827777777777777]
Array of average All Emotions accurecies for user s16
Counter({37.5: 33, 50.0: 26, 62.5: 18, 25.0: 10, 75.0: 8, 12.5: 4, 87.5: 1})
Combined Emotions
[46.5, 0.4208293650793651]
Arousal 67.125
Counter({75.0: 28, 62.5: 27, 50.0: 19, 87.5: 18, 37.5: 4, 25.0: 2, 100.0: 2})
Valence 62.875


Array of average Happy Sad accurecies for user s29
Counter({66.66666666666666: 35, 50.0: 28, 83.33333333333334: 28, 33.33333333333333: 5, 100.0: 4})
Happy Sad
[66.33333333333334, 0.6640547138047138]
Array of average All Emotions accurecies for user s29
Counter({50.0: 36, 37.5: 20, 62.5: 18, 25.0: 13, 75.0: 11, 87.5: 2})
Combined Emotions
[50.0, 0.4542354728604729]
Arousal 72.625
Counter({62.5: 35, 75.0: 35, 87.5: 17, 100.0: 7, 50.0: 4, 25.0: 1, 37.5: 1})
Valence 59.0
Counter({62.5: 31, 50.0: 24, 75.0: 22, 37.5: 11, 25.0: 6, 87.5: 4, 100.0: 2})
Arousal Valence
[72.625, 59.0, 0.693811494061494, 0.586838023088023]
s30
Array of average Happy Sad accurecies for user s30
Counter({57.14285714285714: 36, 42.857142857142854: 29, 71.42857142857143: 23, 28.57142857142857: 5, 85.71428571428571: 5, 14.285714285714285: 2})
Happy Sad
[55.42857142857143, 0.5386985236985237]
Array of average All Emotions accurecies for user s30
Counter({50.0: 41, 37.5: 20, 62.5: 18, 25.0: 15, 75.0: 6})
Combined Emotion